In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#from efficientnet_pytorch import EfficientNet
import xgboost as xgb
from xgboost import XGBClassifier

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
package_path = '../input/efficientnetpytorch/'
sys.path.append(package_path)

from efficientnet_pytorch import EfficientNet

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from __future__ import print_function, absolute_import
import os
import sys
import time
import datetime
import argparse
import os.path as osp
import numpy as np
import random
from PIL import Image
import tqdm
import cv2
import csv
import math
import torchvision as tv
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim import lr_scheduler
from tqdm import tqdm
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
import skimage.io

In [ ]:
name_file='../input/prostate-cancer-grade-assessment/test.csv'
data_dir = '../input/prostate-cancer-grade-assessment'
image_folder = os.path.join(data_dir, 'test_images')
is_test = os.path.exists(image_folder)  # IF test_images is not exists, we will use some train images.
image_folder = image_folder if is_test else os.path.join(data_dir, 'train_images')

csv_file=csv.reader(open(name_file,'r'))
content=[]
for line in csv_file:
    print(line[0]) #打印文件每一行的信息
    content.append(line[0]+'.tiff')
content=content[1:]
if not is_test:
    content.append('002a4db09dad406c85505a00fb6f6144.tiff')  #0
    content.append('007433133235efc27a39f11df6940829.tiff')  #0
    content.append('0018ae58b01bdadc8e347995b69f99aa.tiff')  #4
    content.append('0076bcb66e46fb485f5ba432b9a1fe8a.tiff')  #3
    content.append('003d4dd6bd61221ebc0bfb9350db333f.tiff')  #1
    content=content[3:]
print(content)

In [ ]:
from torch.nn.parameter import Parameter

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
def get_tiles_combine(img,mode=0):
    images = np.ones((1536, 1536, 3))*255
    
    h, w, c = img.shape
    result_all=[]
    pad_h = (256 - h % 256) % 256 + ((256 * mode) // 2)
    pad_w = (256 - w % 256) % 256 + ((256 * mode) // 2)
    #print(pad_h,pad_w,c)
    img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], 'constant',constant_values=255)
    windows=[256,256,256,256,192,192,128]
    x_start=0
    for i in range(len(windows)):
        result = []
        window_size=windows[i]
        for x in range((h+pad_h)//window_size):
            for y in range((w+pad_w)//window_size):
                tile=img2[x*window_size:(x+1)*window_size,y*window_size:(y+1)*window_size]
                result.append([x,y,tile.sum()])
        #print(len(result))
        result.sort(key=lambda ele:ele[2])
        result=result[:1536//window_size]
        #print(len(result),result)
        for y in range(min(1536//window_size,len(result))):
            xx=result[y][0]
            yy=result[y][1]
            result_all.append([xx,yy])
            images[x_start:x_start+window_size,y*window_size:(y+1)*window_size]=\
                img2[xx*window_size:(xx+1)*window_size,yy*window_size:(yy+1)*window_size].copy()
            img2[xx*window_size:(xx+1)*window_size,yy*window_size:(yy+1)*window_size]=255
        x_start=x_start+windows[i]
    return images 
class panda_dataset_combine(Dataset):
    """docstring for data"""
    def __init__(self, txt_path,transform=None):
        imgs = []
        for img in txt_path:
            imgs.append(img)
        self.imgs = imgs
        self.transform = transform
    def __getitem__(self, index):
        fn = self.imgs[index]
        tif_name=fn
        #img = Image.open('./train_images_png/'+fn+'_aug.png').convert('RGB') 
        img_name=tif_name[:-5]
        print(image_folder+'/'+tif_name)
        img=skimage.io.MultiImage(image_folder+'/'+tif_name)[1]
        images=get_tiles_combine(img)
        images2=get_tiles_combine(img,mode=2)
        
        
        #cv2.imwrite('a.png',images)
        #cv2.imwrite('b.png',images2)
        #images = Image.open('a.png').convert('RGB')
        #images2 = Image.open('b.png').convert('RGB')
        #images=cv2.imread('a.png')
        #images2=cv2.imread('b.png')
        #images=cv2.cvtColor(images,cv2.COLOR_BGR2RGB)
        #images2=cv2.cvtColor(images2,cv2.COLOR_BGR2RGB)
        
        
        #images=255-images
        images=images.astype(np.float32)
        images=cv2.cvtColor(images,cv2.COLOR_BGR2RGB)
        #print(images.shape)
        #images = images.transpose(2, 0, 1)
        #images=images/255
        #img=images
        #img = Image.fromarray(images)
        
        #images2=255-images2
        images2=images2.astype(np.float32)
        images2=cv2.cvtColor(images2,cv2.COLOR_BGR2RGB)
        #print(images2.shape)
        #images = images.transpose(2, 0, 1)
        #images2=images2/255
        
        
        images3=cv2.flip(images,0)
        images4=cv2.flip(images2,0)
        images5=cv2.flip(images,1)
        images6=cv2.flip(images2,1)
        #size1=1024
        
        images = Image.fromarray(np.uint8(images))
        images2 = Image.fromarray(np.uint8(images2))
        images3 = Image.fromarray(np.uint8(images3))
        images4 = Image.fromarray(np.uint8(images4))
        images5 = Image.fromarray(np.uint8(images5))
        images6 = Image.fromarray(np.uint8(images6))
        
        if self.transform is not None:
            img = self.transform(images)
            img2= self.transform(images2)
            img3= self.transform(images3)
            img4= self.transform(images4)
            img5= self.transform(images5)
            img6= self.transform(images6)
        #print('ok')
        #return img_name,img,img2
        return img_name,img,img2,img3,img4,img5,img6
    def __len__(self):
        return len(self.imgs)

In [ ]:

tile_size = 256
image_size = 256
n_tiles = 36
batch_size = 8
num_workers = 4

def get_tiles(img, mode=0):
    result = []
    h, w, c = img.shape
    pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
    pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

    img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], 'constant',constant_values=255)
    img3 = img2.reshape(
        img2.shape[0] // tile_size,
        tile_size,
        img2.shape[1] // tile_size,
        tile_size,
        3
    )

    img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
    n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
    if len(img3) < n_tiles:
        img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
    idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
    img3 = img3[idxs]
    for i in range(len(img3)):
        result.append({'img':img3[i], 'idx':i})
    return result, n_tiles_with_info >= n_tiles

class panda_dataset(Dataset):
    """docstring for data"""
    def __init__(self, txt_path,transform=None):
        imgs = []
        for img in txt_path:
            imgs.append(img)
        self.imgs = imgs
        self.transform = transform
    def __getitem__(self, index):
        fn = self.imgs[index]
        tif_name=fn
        #img = Image.open('./train_images_png/'+fn+'_aug.png').convert('RGB') 
        img_name=tif_name[:-5]
        print(image_folder+'/'+tif_name)
        img=skimage.io.MultiImage(image_folder+'/'+tif_name)[1]
        tiles,flag=get_tiles(img)
        tiles2,flag2=get_tiles(img,mode=2)
        idxes = list(range(n_tiles))
        n_row_tiles = int(np.sqrt(n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((image_size, image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img
        images=255-images
        images=images.astype(np.float32)
        images=cv2.cvtColor(images,cv2.COLOR_BGR2RGB)
        print(images.shape)
        #images = images.transpose(2, 0, 1)
        images=images/255
        #img=images
        #img = Image.fromarray(images)
        
        images2 = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
                if len(tiles2) > idxes[i]:
                    this_img = tiles2[idxes[i]]['img']
                else:
                    this_img = np.ones((image_size, image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                h1 = h * image_size
                w1 = w * image_size
                images2[h1:h1+image_size, w1:w1+image_size] = this_img
        images2=255-images2
        images2=images2.astype(np.float32)
        images2=cv2.cvtColor(images2,cv2.COLOR_BGR2RGB)
        print(images2.shape)
        #images = images.transpose(2, 0, 1)
        images2=images2/255
        
        images3=cv2.flip(images,0)
        images4=cv2.flip(images2,0)
        images5=cv2.flip(images,1)
        images6=cv2.flip(images2,1)
        
        
        images_com=get_tiles_combine(img)
        images_com2=get_tiles_combine(img,mode=2)
        
        
        images_com=images_com.astype(np.float32)
        images_com=cv2.cvtColor(images_com,cv2.COLOR_BGR2RGB)

        images_com2=images_com2.astype(np.float32)
        images_com2=cv2.cvtColor(images_com2,cv2.COLOR_BGR2RGB)
        #print(images2.shape)
        #images = images.transpose(2, 0, 1)
        #images2=images2/255
        images_com3=cv2.flip(images_com,0)
        images_com4=cv2.flip(images_com2,0)
        images_com5=cv2.flip(images_com,1)
        images_com6=cv2.flip(images_com2,1)
        #size1=1024
        


        images_com = Image.fromarray(np.uint8(images_com))
        images_com2 = Image.fromarray(np.uint8(images_com2))
        images_com3 = Image.fromarray(np.uint8(images_com3))
        images_com4 = Image.fromarray(np.uint8(images_com4))
        images_com5 = Image.fromarray(np.uint8(images_com5))
        images_com6 = Image.fromarray(np.uint8(images_com6))
        
        
        if self.transform is not None:
            img = self.transform(images)
            img2= self.transform(images2)
            img3= self.transform(images3)
            img4= self.transform(images4)
            img5= self.transform(images5)
            img6= self.transform(images6)
            
            
            img_com=self.transform(images_com)
            img_com2= self.transform(images_com2)
            img_com3= self.transform(images_com3)
            img_com4= self.transform(images_com4)
            img_com5= self.transform(images_com5)
            img_com6= self.transform(images_com6)
            #img3= self.transform(images3)
        #print('ok')
        imgs=[img,img2,img3,img4,img5,img6]
        imgs_com=[img_com,img_com2,img_com3,img_com4,img_com5,img_com6]
        return img_name,imgs,imgs_com
    def __len__(self):
        return len(self.imgs)

In [ ]:
from collections import OrderedDict
def load_para_dict(model1):
    state_dict_1=torch.load(model1)
    new_state_dict = OrderedDict()
    for k, v in state_dict_1.items():
        if 'module' in k:
            name = k[7:] # add `module.`
        else:
            name=k
        new_state_dict[name] = v
    return new_state_dict
def get_preds(arr):
    mask = arr == 0
    return np.clip(np.where(mask.any(1), mask.argmax(1), 6) - 1, 0, 5)

init_coef = [0.5, 1.5, 2.5, 3.5, 4.5]
def get_preds_reg(arr,coef=init_coef):
    X_p = np.copy(arr.detach().cpu())
    for i, pred in enumerate(X_p):
        if pred < coef[0]:
            X_p[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            X_p[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            X_p[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            X_p[i] = 3
        elif pred >= coef[3] and pred < coef[4]:
            X_p[i] = 4
        else:
            X_p[i] = 5
    return X_p


In [ ]:
'''
img=skimage.io.MultiImage(image_folder+'/'+'0a0f8e20b1222b69416301444b117678.tiff')[1]
tiles,flag=get_tiles(img)
tiles2,flag2=get_tiles(img,mode=2)
#tiles2,flag=get_tiles(img,mode=2)
idxes = list(range(n_tiles))
n_row_tiles = int(np.sqrt(n_tiles))
images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
for h in range(n_row_tiles):
    for w in range(n_row_tiles):
        i = h * n_row_tiles + w
        if len(tiles) > idxes[i]:
            this_img = tiles[idxes[i]]['img']
        else:
            this_img = np.ones((image_size, image_size, 3)).astype(np.uint8) * 255
        this_img = 255 - this_img
        h1 = h * image_size
        w1 = w * image_size
        images[h1:h1+image_size, w1:w1+image_size] = this_img
images=255-images

images=images.astype(np.float32)
images=cv2.cvtColor(images,cv2.COLOR_BGR2RGB)
images=images/255
#images = images.transpose(2, 0, 1)
print(images.shape)


images2 = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
for h in range(n_row_tiles):
    for w in range(n_row_tiles):
        i = h * n_row_tiles + w
        if len(tiles2) > idxes[i]:
            this_img = tiles2[idxes[i]]['img']
        else:
            this_img = np.ones((image_size, image_size, 3)).astype(np.uint8) * 255
        this_img = 255 - this_img
        h1 = h * image_size
        w1 = w * image_size
        images2[h1:h1+image_size, w1:w1+image_size] = this_img
images2=255-images2
images2=images2.astype(np.float32)
images2=cv2.cvtColor(images2,cv2.COLOR_BGR2RGB)
print(images2.shape)
#images = images.transpose(2, 0, 1)
images2=images2/255


images3 = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
for h in range(n_row_tiles):
    for w in range(n_row_tiles):
        i = h * n_row_tiles + w
        if len(tiles) > idxes[i]:
            this_img = tiles[idxes[i]]['img']
        else:
            this_img = np.ones((image_size, image_size, 3)).astype(np.uint8) * 255
        this_img = 255 - this_img
        h1 = h * image_size
        w1 = w * image_size
        images3[h1:h1+image_size, w1:w1+image_size] = this_img
images3=255-images3
images3=images3.astype(np.float32)
images3=cv2.cvtColor(images3,cv2.COLOR_BGR2RGB)
images3=cv2.flip(images3,1)
print(images3.shape)
#images = images.transpose(2, 0, 1)
images3=images3/255

import matplotlib.pyplot as plt
plt.figure()
#img = Image.fromarray(images)
print(type(images))
plt.imshow(images)

plt.figure()
plt.imshow(images2)

plt.figure()
plt.imshow(images3)

plt.show()
'''

In [ ]:
from torch.nn.parameter import Parameter

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'
class SwishImplementation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))


class MemoryEfficientSwish(nn.Module):
    def forward(self, x):
        return SwishImplementation.apply(x)

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class isupnet(nn.Module):
    def __init__(self, num_classes,base_model, **kwargs):
        super(isupnet, self).__init__()
        
        
        self.base_model_name=base_model
        self.avg_pooling=GeM(p=3)
        self.max_pooling=nn.AdaptiveMaxPool2d(1)
        if base_model[0]=='e':
            #self.base = EfficientNet.from_pretrained(base_model)
            
            self.base = EfficientNet.from_name(base_model)
            feature = self.base._fc.in_features
        else:
            base_net=torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', 'resnext50_32x4d_ssl')
            feature = base_net.fc.in_features
            self.base=nn.Sequential(*list(base_net.children())[:-2])  
        self.fc = nn.Linear(in_features=2*feature,out_features=feature,bias=True)
        self.classifiers = nn.Linear(in_features=feature, out_features=num_classes)
        self.sigmoid = MemoryEfficientSwish()
        self.dropout=nn.Dropout(0.4)
        self.bn=nn.BatchNorm1d(feature)

    def freeze_base(self):
        for p in self.base.parameters():
            p.requires_grad = False

    def unfreeze_all(self):
        for p in self.parameters():
            p.requires_grad = True
    def forward(self, x1):
        bs=x1.size(0)
        if self.base_model_name[0]=='e':
            x2 = self.base.extract_features(x1)
            feat1=self.avg_pooling(x2)
            feat2=self.max_pooling(x2)
            feat1=feat1.view(bs,-1)
            feat2=feat2.view(bs,-1)
            feat=torch.cat([feat1,feat2],1)
            
            feat = self.fc(feat)
            #feat = self.sigmoid(feat)
            #feat = self.bn(feat)
            feat = self.dropout(feat)
            ys = self.classifiers(feat)

            #ys=self.sigmoid(ys)
        else:
            '''
            x=x1
            x=x.view(bs,3,-1,224,224)
            x=x.permute(0,2,1,3,4).contiguous()
            shape = [36,3,224,224]
            #print(x.shape)
            n = x.shape[1]
            x = x.view(-1,shape[1],shape[2],shape[3])
            #print(x.shape)
            #x: bs*N x 3 x 128 x 128
            x = self.base(x)
            #x: bs*N x C x 4 x 4
            shape = x.shape
            #print(shape)
            #concatenate the output for tiles into a single map
            x = x.view(-1,n,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
              .view(-1,shape[1],shape[2]*n,shape[3])
            #x: bs x C x N*4 x 4
            #print(x.shape)
            '''
            x=self.base(x1)
            feat1=self.avg_pooling(x)
            feat2=self.max_pooling(x)
            feat1=feat1.view(bs,-1)
            feat2=feat2.view(bs,-1)
            feat=torch.cat([feat1,feat2],1)
            
            feat = self.fc(feat)
            #feat = self.sigmoid(feat)
            #feat = self.bn(feat)
            feat = self.dropout(feat)
            ys = self.classifiers(feat)
            #x: bs x n
        

        return ys

In [ ]:

if __name__ == '__main__':

    use_gpu = torch.cuda.is_available()
    if use_gpu:
        cudnn.benchmark = True
        torch.cuda.manual_seed_all(0)
    else:
        print("Currently using CPU (GPU is highly recommended)")


    transform2 = transforms.Compose([
            #transforms.Resize((512, 512)),
            #transforms.ColorJitter(brightness=20,contrast=0.2,saturation=20,hue=0.1),
            transforms.ToTensor(), # 转为Tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # 归一化
                             ]) 



    test_data=panda_dataset(content,transform2)
    #net=Baseline_single(num_classes=5)
    #net = EfficientNet.from_pretrained(args.model)
    net = EfficientNet.from_name('efficientnet-b0')
    feature = net._fc.in_features
    net._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net._avg_pooling=GeM()
    net1 = EfficientNet.from_name('efficientnet-b0')
    feature = net1._fc.in_features
    net1._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net1._avg_pooling=GeM()
    net2 = EfficientNet.from_name('efficientnet-b0')
    feature = net2._fc.in_features
    net2._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net2._avg_pooling=GeM()
    net3 = EfficientNet.from_name('efficientnet-b0')
    feature = net3._fc.in_features
    net3._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net3._avg_pooling=GeM()
    net4 = EfficientNet.from_name('efficientnet-b0')
    feature = net4._fc.in_features
    net4._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net4._avg_pooling=GeM()
    net5 = EfficientNet.from_name('efficientnet-b0')
    feature = net5._fc.in_features
    net5._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net5._avg_pooling=GeM()
    
    net6 = EfficientNet.from_name('efficientnet-b0')
    feature = net6._fc.in_features
    net6._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net6._avg_pooling=GeM()
    
    net7 = EfficientNet.from_name('efficientnet-b0')
    feature = net7._fc.in_features
    net7._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net7._avg_pooling=GeM()
    nets=[net,net1,net2,net3,net4,net5,net6,net7]
    #net8=isupnet(num_classes=6,base_model='efficientnet-b0')
    '''
    net8 = EfficientNet.from_name('efficientnet-b0')
    feature = net8._fc.in_features
    net8._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net8._avg_pooling=GeM()
    '''
    if use_gpu:
        net=net.cuda()
        net1=net1.cuda()
        net2=net2.cuda()
        net3=net3.cuda()
        net4=net4.cuda()
        net5=net5.cuda()
        net6=net6.cuda()
        net7=net7.cuda()
        #net8=net8.cuda()
    net.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_combine_efficientnet-b0_0.00032_adam_4_bce_karmax_gem_1536_augt_845_kar8731.pkl'))
    net1.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.0003_adam_3_bce_maxest_aug_gem_8624.pkl'))
    net2.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.0003_adam_1_bce_maxest_gem_8647.pkl'))
    net3.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.00032_adam_4_bce_maxest_aug_gem_8616.pkl'))
    net4.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.00032_adam_4_bce_maxest_augb_gem_867.pkl'))    
    net5.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_256_efficientnet-b0_0.00032_adam_4_bce_karmax_gem_1536_augb_kar8843_8647.pkl'))    
    net6.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_combine_efficientnet-b0_0.00032_adam_4_bce_karmax_gem_1536_naugt_nbal_kar8928_8725.pkl'))
    net7.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_256_efficientnet-b0_0.00032_adam_0_bce_karmax_gem_1536_augb_kar8741_8567.pkl'))
    #net8.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_combine_efficientnet-b0_0.00032_adam_1_bce_karmax_gem_1536_naugt_nbal_kar8953_8707.pkl'))
    print(net._avg_pooling.p)
    #net._avg_pooling=nn.AdaptiveAvgPool2d(1)
    dataloader_test=DataLoader(
        test_data,batch_size=1, shuffle = False, num_workers= 5)
    idx=0
    max_correct=0
    with open('/kaggle/working/submission.csv',"a+", newline='')as f:
        f_csv = csv.writer(f)
        f_csv.writerow(['image_id','isup_grade'])
    with torch.no_grad():
        net.eval()
        net1.eval()
        net2.eval()
        net3.eval()
        net4.eval()
        net5.eval()
        net6.eval()
        net7.eval()
        #net8.eval()
        total=0
        total_loss=0
        correct=0
        for id,item in enumerate(dataloader_test):
            print('id ',id)            
            name,datas,datas_com=item
            if use_gpu:
                for i in range(len(datas)):
                    datas[i]=datas[i].cuda()
                    datas_com[i]=datas_com[i].cuda()
                #data3=data3.cuda()
            print(datas[0].size())
            outs=[]
            for i in range(len(nets)):
                if i==0 or i== 6:
                    for j in range(len(datas_com)):
                        outs.append(nets[i](datas_com[j]))
                else:
                    for j in range(len(datas_com)):
                        outs.append(nets[i](datas[j]))
            out=outs[0]
            for i in range(1,len(outs)):
                out=out+outs[i]
            
            out=out/len(outs)
            #out=(out3_0+out3_1+out4_1+out4_0)/4
            #out=(out0_0+out0_1+out0_2)/3
            print(out)
            predicted=get_preds((torch.sigmoid(out) > 0.5).cpu().numpy())
            #_, predicted = torch.max(out, 1)
            hh=[str(name[0]),str(predicted[0])]
            print(hh)
            with open('/kaggle/working/submission.csv',"a+", newline='')as f:
                f_csv = csv.writer(f)
                f_csv.writerow(hh)
    print(pd.read_csv('/kaggle/working/submission.csv').isup_grade.value_counts())


In [ ]:
'''
if __name__ == '__main__':

    use_gpu = torch.cuda.is_available()
    if use_gpu:
        cudnn.benchmark = True
        torch.cuda.manual_seed_all(0)
    else:
        print("Currently using CPU (GPU is highly recommended)")


    transform2 = transforms.Compose([
            #transforms.Resize((512, 512)),
            #transforms.ColorJitter(brightness=20,contrast=0.2,saturation=20,hue=0.1),
            transforms.ToTensor(), # 转为Tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # 归一化
                             ]) 



    test_data=panda_dataset(content,transform2)
    net = EfficientNet.from_name('efficientnet-b0')
    feature = net._fc.in_features
    net._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net._avg_pooling=GeM()
    net1 = EfficientNet.from_name('efficientnet-b0')
    feature = net1._fc.in_features
    net1._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net1._avg_pooling=GeM()
    net2 = EfficientNet.from_name('efficientnet-b0')
    feature = net2._fc.in_features
    net2._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net2._avg_pooling=GeM()
    net3 = EfficientNet.from_name('efficientnet-b0')
    feature = net3._fc.in_features
    net3._fc = nn.Linear(in_features=feature,out_features=6,bias=True)
    net3._avg_pooling=GeM()
    if use_gpu:
        net=net.cuda()
        net1=net1.cuda()
        net2=net2.cuda()
        net3=net3.cuda()
    net.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.00032_adam_4_bce_maxest_aug_gem_8711.pkl'))
    net1.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.0003_adam_3_bce_maxest_aug_gem_8624.pkl'))
    net2.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.0003_adam_1_bce_maxest_gem_8647.pkl'))
    net3.load_state_dict(load_para_dict('/kaggle/input/pandamodels/model_36_256_256_efficientnet-b0_0.00032_adam_4_bce_maxest_aug_gem_8616.pkl'))
    print(net3._avg_pooling.p)
    #net._avg_pooling=nn.AdaptiveAvgPool2d(1)
    dataloader_test=DataLoader(
        test_data,batch_size=1, shuffle = False, num_workers= 3)
    idx=0
    max_correct=0
    with open('/kaggle/working/submission.csv',"a+", newline='')as f:
        f_csv = csv.writer(f)
        f_csv.writerow(['image_id','isup_grade'])
    
    with torch.no_grad():
        net.eval()
        net1.eval()
        net2.eval()
        net3.eval()
        total=0
        total_loss=0
        correct=0
        for id,item in enumerate(dataloader_test):
            print('id ',id)            
            name,data,data2=item
            if use_gpu:
                data=data.cuda()
                data2=data2.cuda()
            #print(data.size())
            feature=[]
            out0_0=net(data)
            feature.append(out0_0)
            out0_1=net(data2)
            feature.append(out0_1)
            out1_0=net1(data)
            feature.append(out1_0)
            out1_1=net1(data2)
            feature.append(out1_1)
            out2_0=net2(data)
            feature.append(out2_0)
            out2_1=net2(data2)
            feature.append(out2_1)
            out3_0=net3(data)
            feature.append(out3_0)
            out3_1=net3(data2)
            feature.append(out3_1)
            #out1=net1(data)
            out_fea=feature[0].detach().cpu().numpy()[0]
            for i in range(1,8):
                out_fea=np.hstack((out_fea,feature[i].detach().cpu().numpy()[0]))
                
            out_fea_str=''
            for x in out_fea:
                out_fea_str=out_fea_str+str(x)+'\t'
            out=(out0_0+out0_1+out1_0+out1_1+out2_0+out2_1+out3_0+out3_1)/8
            #out=(out0_0+out0_1+out1_0+out1_1+out2_0+out2_1)/6
            print(out)
            predicted=get_preds((torch.sigmoid(out) > 0.5).cpu().numpy())
            #_, predicted = torch.max(out, 1)
            hh=[name[0],str(predicted[0]),out_fea_str]
            print(hh)
            with open('/kaggle/working/bce_xg_data.csv',"a+", newline='')as f:
                f_csv = csv.writer(f)
                f_csv.writerow(hh)
    bst = xgb.Booster(model_file='/kaggle/input/pandamodels/model_xg.txt')
    bst.save_model('/kaggle/working/model_xg_new.txt')
    bst = xgb.Booster(model_file='/kaggle/working/model_xg_new.txt')
    data=[]
    label=[]
    name=[]
    reader = csv.reader(open('/kaggle/working/bce_xg_data.csv','r'))
    cnt=0

    for line in reader:
        data.append(line[2].split('\t')[:-1])
        label.append(int(line[1]))
        name.append(line[0])
        #print(line[3].split('\t')[:-1])
        cnt=cnt+1
    for x in range(len(data)):
        for y in range(len(data[x])):
            data[x][y]=float(data[x][y])
    valid_X=np.array(data)
    dtest=xgb.DMatrix(valid_X)
    y_pred = bst.predict(dtest)
    ll=y_pred.shape[0]
    for i in range(ll):
        pred=y_pred[i]
        hh=[name[i],str(int(pred))]
        print(hh)
        with open('/kaggle/working/submission.csv',"a+", newline='')as f:
            f_csv = csv.writer(f)
            f_csv.writerow(hh)
'''

In [ ]:
'''
print(type(data),data)
data=np.array(data)
valid_X=data
dtest=xgb.DMatrix(valid_X)
'''
#print(ll)
#print(pd.read_csv('/kaggle/working/submission.csv').isup_grade.value_counts())